# Task 3

## Original exercise number

Exercise 18-6

## Description

The following are the possible hands in poker, in increasing order of value and decreasing order of probability:

*pair*

two cards with the same rank

*two pair*

two pairs of cards with the same rank

*three of a kind*

three cards with the same rank

*straight*

five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.)

*flush*

five cards with the same suit

*full house*

three cards with one rank, two cards with another

*four of a kind*

four cards with the same rank

*straight flush*

five cards in sequence (as defined above) and with the same suit

The goal of this exercise is to estimate the probability of drawing these various hands.

1. Add methods named `haspair`, `hastwopair`, etc. that return `true` or `false` according to whether or not the hand meets the relevant criteria. Your code should work correctly for “hands” that contain any number of cards (although 5 and 7 are the most common sizes).

2. Write a method named classify that figures out the highest-value classification for a hand and sets the label field accordingly. For example, a 7-card hand might contain a flush and a pair; it should be labeled “flush”.

3. When you are convinced that your classification methods are working, the next step is to estimate the probabilities of the various hands. Write a function that shuffles a deck of cards, divides it into hands, classifies the hands, and counts the number of times various classifications appear.

4. Print a table of the classifications and their probabilities. Run your program with larger and larger numbers of hands until the output values converge to a reasonable degree of accuracy. Compare your results to the values at https://en.wikipedia.org/wiki/Hand_rankings.

## Solution

NO GUARANTEE THAT THE SOLUTION WILL WORK OR WORKS CORRECTLY! USE IT AT
YOUR OWN RISK!

### Imports

In [ ]:
import StatsBase as sb

### Structs

In [ ]:
struct Card
    suit :: Int64
    rank :: Int64
    function Card(suit::Int64, rank::Int64)
        @assert(1 ≤ suit ≤ 4, "suit is not between 1 and 4")
        @assert(1 ≤ rank ≤ 13, "rank is not between 1 and 13")
        new(suit, rank)
    end
end

In [ ]:
abstract type CardSet end

In [ ]:
struct Deck <: CardSet
    cards :: Array{Card, 1}
end

function Deck()
    deck = Deck(Card[])
    for suit in 1:4
        for rank in 1:13
            push!(deck.cards, Card(suit, rank))
        end
    end
    deck
end

In [ ]:
struct Hand <: CardSet
    cards :: Array{Card, 1}
    label :: String
end

function Hand(label::String="")
    Hand(Card[], label)
end

### Global Variables

In [ ]:
const suit_names = ["♣", "♦", "♥", "♠"]
const rank_names = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
const classificationIDs2Labels = Dict(
    1 => "straight flush",
    2 => "four of a kind",
    3 => "full house",
    4 => "flush",
    5 =>"straight",
    6 =>"three of a kind",
    7 =>"two pairs",
    8 => "pair",
    9 =>"high card")

### Functions

In [ ]:
function Base.show(io::IO, card::Card)
    print(io, rank_names[card.rank], suit_names[card.suit])
end

In [ ]:
function Base.show(io::IO, cs::CardSet)
    for card in cs.cards
        print(io, card, " ")
    end
end

### Functions (solution to task 3.1)

In [ ]:
function getCounts(v::Vector{T})::Dict{T,Int} where {T}
    result::Dict{T,Int} = Dict()
    for elt in v
        result[elt] = get(result, elt, 0) + 1
    end
    return result
end

function getRanks(cs::CardSet)::Vector{Int}
    return map(c -> c.rank, cs.cards)
end

function getSuits(cs::CardSet)::Vector{Int}
    return map(c -> c.suit, cs.cards)
end

function hasNOfAkind(hand::Hand, n::Int, ranks::Bool = true)::Bool
    @assert 1 < n < 6 "n must be between 2 and 5"
    theCounts::Dict{Int, Int} = getCounts(ranks ? getRanks(hand) : getSuits(hand))
    return any(v -> v == n, values(theCounts))
end

function hasPair(hand::Hand)::Bool
    return hasNOfAkind(hand, 2)
end

function hasTwoPairs(hand::Hand)::Bool
    rankCounts::Dict{Int, Int} = getCounts(getRanks(hand))
    twoPairs::Dict{Int, Int} = Dict(k => v for(k, v) in rankCounts if v == 2)
    return length(twoPairs) == 2
end

function hasThreeOfAKind(hand::Hand)::Bool
    return hasNOfAkind(hand, 3)
end

function hasAKQJ10(h::Hand)::Bool
    theCounts::Dict{Int, Int} = getCounts(getRanks(h))
    for rank in [10, 11, 12, 13, 1]
        if !haskey(theCounts, rank)
            return false
        end
    end
    return true
end

function isInAscOrder(v::Vector{Int})::Bool
    vectInConseqOrd::UnitRange{Int} = v[1]:v[1]+length(v)-1
    return all(map((x, y) -> x == y, v, vectInConseqOrd))
end

function isInAscOrder(v::Vector{Int}, lenInOrder::Int)::Bool
    if length(v) < lenInOrder
        return false
    end
    for i in 1:length(v)-(lenInOrder-1)
        if isInAscOrder(v[i:i+(lenInOrder-1)])
            return true
        end
    end
    return false
end

function hasStraight(h::Hand)::Bool
    return hasAKQJ10(h) || isInAscOrder(sort(getRanks(h)), 5)
end

function hasFlush(hand::Hand)::Bool
    return hasNOfAkind(hand, 5, false)
end

function hasFullHouse(hand::Hand)::Bool
    return hasPair(hand) && hasThreeOfAKind(hand)
end

function hasFourOfAKind(hand::Hand)::Bool
    return hasNOfAkind(hand, 4)
end

function getDominantSuit(hand::Hand)::Int
    suitsCounts::Dict{Int, Int} = getCounts(getSuits(hand))
    suitsMode::Int = maximum(values(suitsCounts)) 
    return [k for (k, v) in suitsCounts if v == suitsMode][1]
end

function hasStraightFlush(hand::Hand)::Bool
    if hasFlush(hand)
        dominantSuit::Int = getDominantSuit(hand)
        cards::Vector{Card} = [c for c in hand.cards if c.suit == dominantSuit]
        return hasStraight(Hand(cards, ""))
    else
        return false
    end
end

### Functions (solution to task 3.2)

In [ ]:
function classify(hand)::Int
    return (
        hasStraightFlush(hand) ? 1 :
        hasFourOfAKind(hand) ? 2 :
        hasFullHouse(hand) ? 3 :
        hasFlush(hand) ? 4 :
        hasStraight(hand) ? 5 :
        hasThreeOfAKind(hand) ? 6 :
        hasTwoPairs(hand) ? 7 :
        hasPair(hand) ? 8 :
        9
    )
end

### Functions (solution to task 3.3 and task 3.4)

In [ ]:
function getRandHand(deck::Deck, cardsInHand::Int=5)::Hand
    return Hand(sb.sample(deck.cards, cardsInHand; replace=false), "")
end

function getNHandsClassifications(n::Int=1_000_000, cardsInHand::Int=5)::Vector{Int}
    @assert n > 0 "n must be positive integer"
    result::Vector{Int} = zeros(n)
    deck::Deck = Deck()
    for i in 1:n
        result[i] = classify(getRandHand(deck, cardsInHand))
    end
    return result
end

function getProbabilities(counts::Dict{T, Int})::Dict{T, Float64} where T
    total::Int = sum(values(counts))
    return Dict(k => v/total for (k, v) in counts)
end


## Testing

### Five Card poker hands

Compare with a table found here: https://en.wikipedia.org/wiki/Poker_probability#5-card_poker_hands 

In [ ]:
hands5classifications = getNHandsClassifications(1_000_000, 5)
hands5counts = getCounts(hands5classifications)
hands5probabilities = getProbabilities(hands5counts)

for (k, v) in sort(hands5probabilities)
    println("$(classificationIDs2Labels[k]): $(round(v*100, digits=5))%")
end

### Seven Card poker hands

Compare with a table found here: https://en.wikipedia.org/wiki/Poker_probability#7-card_poker_hands

In [ ]:
hands7classifications = getNHandsClassifications(100_000, 7)
hands7counts = getCounts(hands7classifications)
hands7probabilities = getProbabilities(hands7counts)

for (k, v) in sort(hands7probabilities)
    println("$(classificationIDs2Labels[k]): $(round(v*100, digits=5))%")
end

#### Note

All the probabilities calculated for 5 card poker hands seem to be OK. 

In the case of seven card poker hands the probability for straight seems to be a bit too low (the rest of the estimates are more or less OK).

I'm not sure why, the code for `hasStraingt` seems to be written correctly (the way I understand the rules based on the task description).

Perhaps "(aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not)" from the task description means that, e.g. Ace-5-6-7-8 or 7-8-9-10-Ace should be also treated as Straights? But that would be weird. Since it is just an exercise I think I'll stop digging and leave it as it is.